## Annotate Data

In [1]:
import pandas as pd
import os
import glob
from collections import Counter
from matplotlib.pyplot import scatter
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
%matplotlib notebook

In [3]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

### 1. Annotate those that need to be removed from analysis based on QC Data Results

In [4]:
## Read in the data 
recgli = pd.read_csv('../GetMergeData/final_mri_with_path.csv', index_col = 0)

In [5]:
## Read in the QC
qc = pd.read_csv('../QCData/final_biopsy_qc.csv')

In [6]:
print(qc.shape, recgli.shape)

(380, 10) (479, 54)


In [7]:
Counter(qc.comments)

Counter({'n': 329,
         'weird place': 2,
         nan: 4,
         'bad perf': 6,
         'change outside to y - very bad': 1,
         'VERY BAD PERF': 2,
         'VERY BAD PErF': 1,
         'almost outside might exclude': 1,
         'right in black hole exclude': 1,
         'right in black hole': 1,
         'right in black spot exclude': 1,
         'dark spot not black': 1,
         'exclude bad perf': 4,
         'y distortion': 1,
         'artifact': 1,
         'y': 1,
         'very close to sus artifact': 1,
         'dark spot': 1,
         'corpus': 1,
         'bad_perf': 1,
         'in black spot? ': 1,
         'y outside perf': 1,
         'all black': 1,
         'almost at edge of hematoma/outside perf exclude': 1,
         "switch to n's": 1,
         'switch to y outside perf': 1,
         'exclude': 4,
         'm': 7,
         'notworkin': 1,
         'nb2571': 1})

In [8]:
## Removing the NA's from the comments
qc_noxs  = qc.dropna(subset = ['comments'], axis = 0)
qc = qc_noxs
qc.shape

(376, 10)

In [9]:
recgli.head()

,roi.label,b_number,t_number_x,t_number_y,sf_number,current_scan_date,current_surgery_date,current_hist_grade,current_hist_type,tumor_cell_evaluation,necrosis,mv_hyperplasia,mv_delicate_vasculature,f8_delicate,f8_simple,f8_complex,mib_1,avgerage_cells,treatment_related_ab,bx_pure_treatment_effect,percent_tumor_nuclei,nfl,nfse,nt1c,nt1v,nt1d,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1,nadc.2,nfa.2,nev1.2,nev2.2,nev3.2,nevrad.2,cbvn_nlin,phn_nlin,recov_nlin,phn_npar,recov_npar,recovn_npar,ccri,cni,crni,ncho,ncre,nnaa,nlac,nlip,newdata,olddata
0,10B28,2723.0,t6369,6369.0,8122.0,2009-09-13,2009-09-14,Grade IV,Glioblastoma,0,0,0,1,2,1,0,0.82,183.75,1,t,NaN,1.84,2.16,1.22,0.86,0.24,1.20,0.81,1.03,1.10,1.20,1.09,NaN,NaN,NaN,NaN,NaN,NaN,0.66,0.81,84.53,0.79,84.71,NaN,2.46,2.44,-0.25,1.21,0.39,0.53,0.37,7.29,0,1
1,11B16,2312.0,t5317,5317.0,7679.0,2008-05-04,2008-05-04,Grade IV,Glioblastoma,2,1,1,1,2,3,0,3.31,146.80,0,f,NaN,1.44,2.25,1.98,0.73,1.30,1.87,0.46,1.41,1.68,1.76,1.61,0.00,0.00,0.00,0.00,0.00,0.0,2.24,2.25,84.88,2.25,87.55,NaN,0.30,1.84,0.97,0.78,0.74,0.36,0.42,21.54,0,1
2,11B40,2474.0,t7408,7408.0,8701.0,2011-03-23,2011-03-24,Grade IV,Glioblastoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.97,4.13,0.57,0.95,0.28,0.08,-0.03,-2.71,0,1
3,11B43,3169.0,t7540,7540.0,8796.0,2011-06-19,2011-06-20,Grade IV,Glioblastoma,2,0,0,NaN,3,0,0,17.00,234.60,0,f,NaN,2.01,2.40,1.00,0.82,-0.01,1.39,0.65,1.12,1.11,1.16,1.13,1.49,0.83,1.12,1.31,1.39,1.4,0.58,0.67,100.00,0.60,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
4,12B16,2312.0,t5317,5317.0,7679.0,2008-05-04,2008-05-04,Grade IV,Glioblastoma,2,1,1,0,3,2,0,12.84,185.00,0,f,NaN,1.53,2.34,2.04,0.77,1.33,1.90,0.46,1.41,1.72,1.76,1.61,0.00,0.00,0.00,0.00,0.00,0.0,1.71,1.73,85.03,1.88,88.16,NaN,1.20,2.04,0.37,1.18,0.89,0.73,0.62,8.14,0,1


In [10]:
recgli.columns

Index(['roi.label', 'b_number', 't_number_x', 't_number_y', 'sf_number',
       'current_scan_date', 'current_surgery_date', 'current_hist_grade',
       'current_hist_type', 'tumor_cell_evaluation', 'necrosis',
       'mv_hyperplasia', 'mv_delicate_vasculature', 'f8_delicate', 'f8_simple',
       'f8_complex', 'mib_1', 'avgerage_cells', 'treatment_related_ab',
       'bx_pure_treatment_effect', 'percent_tumor_nuclei', 'nfl', 'nfse',
       'nt1c', 'nt1v', 'nt1d', 'nadc.1', 'nfa.1', 'nev1.1', 'nev2.1', 'nev3.1',
       'nevrad.1', 'nadc.2', 'nfa.2', 'nev1.2', 'nev2.2', 'nev3.2', 'nevrad.2',
       'cbvn_nlin', 'phn_nlin', 'recov_nlin', 'phn_npar', 'recov_npar',
       'recovn_npar', 'ccri', 'cni', 'crni', 'ncho', 'ncre', 'nnaa', 'nlac',
       'nlip', 'newdata', 'olddata'],
      dtype='object')

In [11]:
recgli.columns = ['vialid', 'b_number', 't_number_x', 't_number_y', 'sf_number',
       'current_scan_date', 'current_surgery_date', 'current_hist_grade',
       'current_hist_type', 'tumor_cell_evaluation', 'necrosis',
       'mv_hyperplasia', 'mv_delicate_vasculature', 'f8_delicate', 'f8_simple',
       'f8_complex', 'mib_1', 'avgerage_cells', 'treatment_related_ab',
       'bx_pure_treatment_effect', 'percent_tumor_nuclei', 'nfl', 'nfse',
       'nt1c', 'nt1v', 'nt1d', 'nadc.1', 'nfa.1', 'nev1.1', 'nev2.1', 'nev3.1',
       'nevrad.1', 'nadc.2', 'nfa.2', 'nev1.2', 'nev2.2', 'nev3.2', 'nevrad.2',
       'cbvn_nlin', 'phn_nlin', 'recov_nlin', 'phn_npar', 'recov_npar',
       'recovn_npar', 'ccri', 'cni', 'crni', 'ncho', 'ncre', 'nnaa', 'nlac',
       'nlip', 'newdata', 'olddata']

In [12]:
def exclude_completely(row): 
    exclude_status = 0
    if row['hematoma']=="y": 
        exclude_status = 1
    elif row['ventricle']=='y': 
        exclude_status = 1
    elif 'exclude' in row['comments'].split(): 
        exclude_status = 1
    return exclude_status

In [13]:
def find_exclusions(dataframe):
    exclude_list = []
    for idx, row in dataframe.iterrows(): 
        row_status = exclude_completely(row)
        exclude_list.append(row_status)
    return exclude_list 

In [14]:
def exclude_perf(row): 
    exclude_perf_status = 0
    if row['outside_perf']=="y": 
        exclude_perf_status = 1
    elif 'bad' in row['comments'].split(): 
        exclude_perf_status = 1
    return exclude_perf_status

In [15]:
def find_perf_exclusions(dataframe):
    perf_exclude_list = []
    for idx, row in dataframe.iterrows(): 
        row_status = exclude_perf(row)
        perf_exclude_list.append(row_status)
    return perf_exclude_list 

In [16]:
## Adding in exclusion of biopsy completely based on inside hematoma or inside ventricle: 
qc['exclude_bc_hematoma_or_ventricle']= find_exclusions(qc)

In [17]:
## Adding in exclusion of perfusion 
qc['exclude_perf'] = find_perf_exclusions(qc)

In [18]:
## Combining this data with that of the biopsy data from recgli: 
recgli_qc = pd.merge(recgli, qc[['vialid', 'exclude_bc_hematoma_or_ventricle', 'exclude_perf']], how = 'left', on = 'vialid')

In [19]:
recgli_qc.to_csv('recgli_annotation1_qc.csv', index = False)
recgli_qc.shape

(479, 56)

In [20]:
recgli_qc.head()

,vialid,b_number,t_number_x,t_number_y,sf_number,current_scan_date,current_surgery_date,current_hist_grade,current_hist_type,tumor_cell_evaluation,necrosis,mv_hyperplasia,mv_delicate_vasculature,f8_delicate,f8_simple,f8_complex,mib_1,avgerage_cells,treatment_related_ab,bx_pure_treatment_effect,percent_tumor_nuclei,nfl,nfse,nt1c,nt1v,nt1d,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1,nadc.2,nfa.2,nev1.2,nev2.2,nev3.2,nevrad.2,cbvn_nlin,phn_nlin,recov_nlin,phn_npar,recov_npar,recovn_npar,ccri,cni,crni,ncho,ncre,nnaa,nlac,nlip,newdata,olddata,exclude_bc_hematoma_or_ventricle,exclude_perf
0,10B28,2723.0,t6369,6369.0,8122.0,2009-09-13,2009-09-14,Grade IV,Glioblastoma,0,0,0,1,2,1,0,0.82,183.75,1,t,NaN,1.84,2.16,1.22,0.86,0.24,1.20,0.81,1.03,1.10,1.20,1.09,NaN,NaN,NaN,NaN,NaN,NaN,0.66,0.81,84.53,0.79,84.71,NaN,2.46,2.44,-0.25,1.21,0.39,0.53,0.37,7.29,0,1,0.0,0.0
1,11B16,2312.0,t5317,5317.0,7679.0,2008-05-04,2008-05-04,Grade IV,Glioblastoma,2,1,1,1,2,3,0,3.31,146.80,0,f,NaN,1.44,2.25,1.98,0.73,1.30,1.87,0.46,1.41,1.68,1.76,1.61,0.00,0.00,0.00,0.00,0.00,0.0,2.24,2.25,84.88,2.25,87.55,NaN,0.30,1.84,0.97,0.78,0.74,0.36,0.42,21.54,0,1,0.0,0.0
2,11B40,2474.0,t7408,7408.0,8701.0,2011-03-23,2011-03-24,Grade IV,Glioblastoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.97,4.13,0.57,0.95,0.28,0.08,-0.03,-2.71,0,1,NaN,NaN
3,11B43,3169.0,t7540,7540.0,8796.0,2011-06-19,2011-06-20,Grade IV,Glioblastoma,2,0,0,NaN,3,0,0,17.00,234.60,0,f,NaN,2.01,2.40,1.00,0.82,-0.01,1.39,0.65,1.12,1.11,1.16,1.13,1.49,0.83,1.12,1.31,1.39,1.4,0.58,0.67,100.00,0.60,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0.0,0.0
4,12B16,2312.0,t5317,5317.0,7679.0,2008-05-04,2008-05-04,Grade IV,Glioblastoma,2,1,1,0,3,2,0,12.84,185.00,0,f,NaN,1.53,2.34,2.04,0.77,1.33,1.90,0.46,1.41,1.72,1.76,1.61,0.00,0.00,0.00,0.00,0.00,0.0,1.71,1.73,85.03,1.88,88.16,NaN,1.20,2.04,0.37,1.18,0.89,0.73,0.62,8.14,0,1,0.0,0.0


In [21]:
recgli_qc = recgli_qc.drop_duplicates('vialid')
recgli_qc.shape

(479, 56)

## 2. Annotate the data for lactate+lipid combination: 

In [22]:
## If the lactate = lipid, then it's the combination of the two, 
## if lactate != lipid, then they need to be added together to find combo Lac+Lip
# def determine_nLL(row): 
#     if row['nlac']==row['nlip']: 
#         nLL = row['nlac']
#     else: 
#         nLL = row['nlac']+row['nlip']

def find_nLL(dataframe): 
    nLL_list = []
    for idx, row in dataframe.iterrows(): 
        if row['nlac']==row['nlip']: 
            nLL = row['nlac']
        else: 
            nLL = row['nlac']+row['nlip']
        nLL_list.append(nLL)
    return nLL_list

In [23]:
recgli_qc['nLL']=find_nLL(recgli_qc)

In [24]:
recgli_qc.tail()

,vialid,b_number,t_number_x,t_number_y,sf_number,current_scan_date,current_surgery_date,current_hist_grade,current_hist_type,tumor_cell_evaluation,necrosis,mv_hyperplasia,mv_delicate_vasculature,f8_delicate,f8_simple,f8_complex,mib_1,avgerage_cells,treatment_related_ab,bx_pure_treatment_effect,percent_tumor_nuclei,nfl,nfse,nt1c,nt1v,nt1d,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1,nadc.2,nfa.2,nev1.2,nev2.2,nev3.2,nevrad.2,cbvn_nlin,phn_nlin,recov_nlin,phn_npar,recov_npar,recovn_npar,ccri,cni,crni,ncho,ncre,nnaa,nlac,nlip,newdata,olddata,exclude_bc_hematoma_or_ventricle,exclude_perf,nLL
474,88B63,1837.0,t8960,8960.0,9913.0,2014-03-18,2014-03-20,Grade IV,Glioblastoma,2,0.0,1,NaN,NaN,NaN,NaN,37.12,264.00,NaN,f,75.0,1.69,1.55,1.30,0.57,0.72,2.30,0.94,1.92,2.23,2.08,2.13,2.13,0.83,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,0,0.0,0.0,0.00
475,89B63,1837.0,t8960,8960.0,9913.0,2014-03-18,2014-03-20,Grade IV,Glioblastoma,3,0.0,2,NaN,NaN,NaN,NaN,49.16,252.00,NaN,f,95.0,1.85,1.79,1.47,0.59,0.87,2.25,0.83,1.84,2.17,2.08,2.10,2.10,0.72,NaN,NaN,NaN,NaN,0.92,0.75,100.00,1.10,100.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,0,0.0,0.0,0.00
476,90B80,4009.0,NaN,10269.0,10723.0,2016-02-16,2016-02-17,Grade III,Oligodendroglioma,3,0.0,0,NaN,NaN,NaN,NaN,16.22,448.00,NaN,f,85.0,1.84,2.08,0.88,0.71,0.16,1.01,1.17,0.91,0.93,0.98,0.90,1.01,0.94,0.85,0.88,1.02,0.94,NaN,NaN,NaN,NaN,NaN,NaN,-1.59,6.78,6.85,1.75,2.13,0.27,0.18,0.18,1,0,NaN,NaN,0.18
477,9B71,3519.0,t9119,9119.0,10064.0,2014-08-04,2014-08-05,Grade IV,Glioblastoma,3,0.0,1,NaN,NaN,NaN,NaN,24.25,364.00,NaN,f,80.0,1.59,1.55,1.70,0.82,0.84,1.79,0.58,1.41,1.77,2.03,1.80,1.84,0.61,1.38,1.67,1.81,1.80,1.16,1.77,100.00,1.53,100.00,100.00,1.18,1.46,0.34,0.63,0.29,0.20,0.51,0.87,1,0,0.0,0.0,1.38
478,9B92,4193.0,t10790,10790.0,11065.0,2016-11-22,2016-11-23,Grade IV,Glioblastoma,2,0.0,0,NaN,NaN,NaN,NaN,4.17,131.75,NaN,t,50.0,1.35,1.46,0.96,0.92,-0.00,1.32,0.50,1.03,1.28,1.44,1.33,1.38,0.72,1.01,1.25,1.36,1.21,0.89,0.80,89.19,0.98,94.15,98.59,-0.70,0.26,0.93,0.75,0.97,0.67,0.51,0.51,1,0,0.0,0.0,0.51


In [25]:
recgli_qc.to_csv('recgli_annotation2_LL.csv', index = False)
recgli_qc.shape

(479, 57)

## 3. Read in path comments for outcome determination: 

In [26]:
oldpathcomments     = pd.read_csv("path_comments_oldP01.csv")
newpathcomments     = pd.read_csv("path_comments_RECHGG.csv")
missingpathcomments = pd.read_csv('missing_path_comments.csv')


In [27]:
oldpathcomments.columns = ['vial_id', 'comments']
newpathcomments.columns = ['vial_id', 'comments']

In [28]:
missingpathcomments = missingpathcomments.dropna(subset = ['vial_id'])
oldpathcomments     = oldpathcomments.dropna(subset = ['vial_id'])
newpathcomments     = newpathcomments.dropna(subset = ['vial_id'])

In [29]:
print(missingpathcomments.shape)
print(oldpathcomments.shape)
print(newpathcomments.shape)

(28, 2)
(371, 2)
(303, 2)


In [30]:
comments = oldpathcomments.append(missingpathcomments)
comments = comments.append(newpathcomments)
comments.shape

(702, 2)

In [31]:
comments = comments.dropna()
comments.shape
comments.columns = ['vialid', 'comments']

In [32]:
comments.head()

,vialid,comments
4,87B44,single cell invasion in cortex
10,25B43,90% BV wall
13,6B43,focally 4.6%
17,82B41,blood
19,13B41,ki67=immune


In [33]:
recgli_qc_withcomments = pd.merge(recgli_qc, comments, how = 'left', on= 'vialid')
recgli_qc_withcomments = recgli_qc_withcomments.drop_duplicates('vialid')

In [34]:
recgli_qc_withcomments.to_csv('recgli_annotation3_pathcomments.csv', index = False)
recgli_qc_withcomments.shape

(479, 58)

## 4. Substitution of 0's for NA's, Annotate those substitutions

#### Perfusion: 

In [35]:
## for brevity, re-assigning recgli_qc_withcomments to recgli: 
recgli = recgli_qc_withcomments

In [36]:
## Zero's mess up averages; find where quant failed 

recgli.loc[(recgli['cbvn_nlin'] == 0) & (recgli['phn_npar']==0), 'perf_quant']='all_zeros'
recgli.loc[(recgli['cbvn_nlin'].isnull()) & (recgli['phn_npar'].isnull()), 'perf_quant']='all_nas'
recgli.loc[(recgli['cbvn_nlin'] > 10) | (recgli['phn_nlin']>10) | (recgli['phn_npar']>10), 'perf_quant']='extreme_value'


In [37]:
recgli.loc[(recgli['perf_quant'].notnull() |  recgli['exclude_perf']==1),   'include_perf']=0
recgli.loc[(recgli['perf_quant'].isnull()  |  recgli['exclude_perf']==0),   'include_perf']=1


In [38]:
recgli.loc[(recgli['perf_quant']=='all_zeros'), ['cbvn_nlin', 'phn_nlin', 'recov_nlin', 'phn_npar', 'recov_npar', 'recovn_npar']]= None
recgli.loc[(recgli['perf_quant']=='extreme_value'), ['cbvn_nlin', 'phn_nlin', 'recov_nlin', 'phn_npar', 'recov_npar', 'recovn_npar']]= None

#### Spectroscopy: 

In [39]:
recgli.columns

Index(['vialid', 'b_number', 't_number_x', 't_number_y', 'sf_number',
       'current_scan_date', 'current_surgery_date', 'current_hist_grade',
       'current_hist_type', 'tumor_cell_evaluation', 'necrosis',
       'mv_hyperplasia', 'mv_delicate_vasculature', 'f8_delicate', 'f8_simple',
       'f8_complex', 'mib_1', 'avgerage_cells', 'treatment_related_ab',
       'bx_pure_treatment_effect', 'percent_tumor_nuclei', 'nfl', 'nfse',
       'nt1c', 'nt1v', 'nt1d', 'nadc.1', 'nfa.1', 'nev1.1', 'nev2.1', 'nev3.1',
       'nevrad.1', 'nadc.2', 'nfa.2', 'nev1.2', 'nev2.2', 'nev3.2', 'nevrad.2',
       'cbvn_nlin', 'phn_nlin', 'recov_nlin', 'phn_npar', 'recov_npar',
       'recovn_npar', 'ccri', 'cni', 'crni', 'ncho', 'ncre', 'nnaa', 'nlac',
       'nlip', 'newdata', 'olddata', 'exclude_bc_hematoma_or_ventricle',
       'exclude_perf', 'nLL', 'comments', 'perf_quant', 'include_perf'],
      dtype='object')

In [40]:
## Zero's mess up averages; find where quant failed 

recgli.loc[(recgli['cni'] == 0) & (recgli['ccri']==0) & (recgli['ncho']==0), 'spec_quant']='all_zeros'
recgli.loc[(recgli['cni'].isnull()) & (recgli['ccri'].isnull()), 'spec_quant']='all_nas'
recgli.loc[(recgli['cni'] > 30) | (recgli['ccri']>30) | (recgli['ncho']>30), 'spec_quant']='extreme_value'


In [41]:
## Find where spec_quant isn't null & exclude from analysis
recgli.loc[(recgli['spec_quant'].notnull() ), 'include_spec']=0
recgli.loc[(recgli['spec_quant'].isnull() ),  'include_spec']=1

In [42]:
## Find where the zero's were and replace w/ NAs

recgli.loc[(recgli['spec_quant']=='all_zeros'), ['ccri', 'cni', 'crni', 'ncho', 'ncre', 'nnaa', 'nlac','nlip', 'nLL']]= None
recgli.loc[(recgli['spec_quant']=='extreme_value'), ['ccri', 'cni', 'crni', 'ncho', 'ncre', 'nnaa', 'nlac','nlip', 'nLL']]= None


In [43]:
recgli = recgli.drop_duplicates('vialid')

#### Diffusion: 

In [44]:
## Zero's mess up averages; find where quant failed 

recgli.loc[(recgli['nadc.1'] == 0) & (recgli['nev1.1']==0), 'diffu1000_quant']='all_zeros'
recgli.loc[(recgli['nadc.2'] == 0) & (recgli['nev1.2']==0), 'diffu2000_quant']='all_zeros'

recgli.loc[(recgli['nadc.1'].isnull()) & (recgli['nev1.1'].isnull()), 'diffu1000_quant']='all_nas'
recgli.loc[(recgli['nadc.2'].isnull()) & (recgli['nev1.2'].isnull()), 'diffu2000_quant']='all_nas'


In [45]:
recgli.loc[(recgli['diffu1000_quant']=='all_zeros'), ['nadc.1', 'nfa.1', 'nev1.1', 'nev2.1', 'nev3.1',
       'nevrad.1']]= None
recgli.loc[(recgli['diffu2000_quant']=='all_zeros'), ['nadc.2', 'nfa.2', 'nev1.2', 'nev2.2', 'nev3.2',
       'nevrad.2']]= None

In [46]:
recgli.columns

Index(['vialid', 'b_number', 't_number_x', 't_number_y', 'sf_number',
       'current_scan_date', 'current_surgery_date', 'current_hist_grade',
       'current_hist_type', 'tumor_cell_evaluation', 'necrosis',
       'mv_hyperplasia', 'mv_delicate_vasculature', 'f8_delicate', 'f8_simple',
       'f8_complex', 'mib_1', 'avgerage_cells', 'treatment_related_ab',
       'bx_pure_treatment_effect', 'percent_tumor_nuclei', 'nfl', 'nfse',
       'nt1c', 'nt1v', 'nt1d', 'nadc.1', 'nfa.1', 'nev1.1', 'nev2.1', 'nev3.1',
       'nevrad.1', 'nadc.2', 'nfa.2', 'nev1.2', 'nev2.2', 'nev3.2', 'nevrad.2',
       'cbvn_nlin', 'phn_nlin', 'recov_nlin', 'phn_npar', 'recov_npar',
       'recovn_npar', 'ccri', 'cni', 'crni', 'ncho', 'ncre', 'nnaa', 'nlac',
       'nlip', 'newdata', 'olddata', 'exclude_bc_hematoma_or_ventricle',
       'exclude_perf', 'nLL', 'comments', 'perf_quant', 'include_perf',
       'spec_quant', 'include_spec', 'diffu1000_quant', 'diffu2000_quant'],
      dtype='object')

In [47]:
recgli.to_csv('recgli_annotation4_excludePerfSpec.csv', index = False)

## 5. Inclusion Binary Codes

In [48]:
recgli = pd.read_csv('recgli_annotation4_excludePerfSpec.csv')
recgli.head()

,vialid,b_number,t_number_x,t_number_y,sf_number,current_scan_date,current_surgery_date,current_hist_grade,current_hist_type,tumor_cell_evaluation,necrosis,mv_hyperplasia,mv_delicate_vasculature,f8_delicate,f8_simple,f8_complex,mib_1,avgerage_cells,treatment_related_ab,bx_pure_treatment_effect,percent_tumor_nuclei,nfl,nfse,nt1c,nt1v,nt1d,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1,nadc.2,nfa.2,nev1.2,nev2.2,nev3.2,nevrad.2,cbvn_nlin,phn_nlin,recov_nlin,phn_npar,recov_npar,recovn_npar,ccri,cni,crni,ncho,ncre,nnaa,nlac,nlip,newdata,olddata,exclude_bc_hematoma_or_ventricle,exclude_perf,nLL,comments,perf_quant,include_perf,spec_quant,include_spec,diffu1000_quant,diffu2000_quant
0,10B28,2723.0,t6369,6369.0,8122.0,2009-09-13,2009-09-14,Grade IV,Glioblastoma,0,0,0,1,2,1,0,0.82,183.75,1,t,NaN,1.84,2.16,1.22,0.86,0.24,1.20,0.81,1.03,1.10,1.20,1.09,NaN,NaN,NaN,NaN,NaN,NaN,0.66,0.81,84.53,0.79,84.71,NaN,2.46,2.44,-0.25,1.21,0.39,0.53,0.37,7.29,0,1,0.0,0.0,7.66,NaN,NaN,NaN,NaN,1.0,NaN,all_nas
1,11B16,2312.0,t5317,5317.0,7679.0,2008-05-04,2008-05-04,Grade IV,Glioblastoma,2,1,1,1,2,3,0,3.31,146.80,0,f,NaN,1.44,2.25,1.98,0.73,1.30,1.87,0.46,1.41,1.68,1.76,1.61,NaN,NaN,NaN,NaN,NaN,NaN,2.24,2.25,84.88,2.25,87.55,NaN,0.30,1.84,0.97,0.78,0.74,0.36,0.42,21.54,0,1,0.0,0.0,21.96,NaN,NaN,NaN,NaN,1.0,NaN,all_zeros
2,11B40,2474.0,t7408,7408.0,8701.0,2011-03-23,2011-03-24,Grade IV,Glioblastoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.97,4.13,0.57,0.95,0.28,0.08,-0.03,-2.71,0,1,NaN,NaN,-2.74,NaN,all_nas,1.0,NaN,1.0,all_nas,all_nas
3,11B43,3169.0,t7540,7540.0,8796.0,2011-06-19,2011-06-20,Grade IV,Glioblastoma,2,0,0,NaN,3,0,0,17.00,234.60,0,f,NaN,2.01,2.40,1.00,0.82,-0.01,1.39,0.65,1.12,1.11,1.16,1.13,1.49,0.83,1.12,1.31,1.39,1.4,0.58,0.67,100.00,0.60,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0.0,0.0,NaN,NaN,NaN,NaN,all_nas,0.0,NaN,NaN
4,12B16,2312.0,t5317,5317.0,7679.0,2008-05-04,2008-05-04,Grade IV,Glioblastoma,2,1,1,0,3,2,0,12.84,185.00,0,f,NaN,1.53,2.34,2.04,0.77,1.33,1.90,0.46,1.41,1.72,1.76,1.61,NaN,NaN,NaN,NaN,NaN,NaN,1.71,1.73,85.03,1.88,88.16,NaN,1.20,2.04,0.37,1.18,0.89,0.73,0.62,8.14,0,1,0.0,0.0,8.76,PP necrosis; some hyalinized BV; tissue is 2/3...,NaN,NaN,NaN,1.0,NaN,all_zeros


In [49]:
recgli.loc[(recgli['nfse'].isnull()), 'exclude_bc_anat']=1
recgli.loc[(recgli['tumor_cell_evaluation'].isnull()), 'exclude_bc_pathology']=1
recgli.loc[(recgli['tumor_cell_evaluation']=="Indeterminable"), 'exclude_bc_pathology']=1

In [50]:
recgli.loc[(recgli['nadc.1'].notnull()), 'include_diffu1000']=1
recgli.loc[(recgli['nadc.1'].isnull()) , 'include_diffu1000']=0

recgli.loc[(recgli['nadc.2'].notnull()), 'include_diffu2000']=1
recgli.loc[(recgli['nadc.2'].isnull()) , 'include_diffu2000']=0


In [51]:
recgli.to_csv('recgli_annotation5_include.csv', index = False)

In [52]:
recgli.tumor_cell_evaluation.value_counts()

3                 140
2                 127
0                 105
1                  58
Indeterminable     13
Name: tumor_cell_evaluation, dtype: int64

In [53]:
recgli.exclude_bc_pathology.value_counts()

1.0    49
Name: exclude_bc_pathology, dtype: int64

In [54]:
recgli.exclude_bc_anat.value_counts()

1.0    27
Name: exclude_bc_anat, dtype: int64

## 6. Since diffusion2000 gives the same as diffusion1000, we will just substitute those in for one another: 

In [55]:
recgli = pd.read_csv('recgli_annotation5_include.csv')
recgli.head()

,vialid,b_number,t_number_x,t_number_y,sf_number,current_scan_date,current_surgery_date,current_hist_grade,current_hist_type,tumor_cell_evaluation,necrosis,mv_hyperplasia,mv_delicate_vasculature,f8_delicate,f8_simple,f8_complex,mib_1,avgerage_cells,treatment_related_ab,bx_pure_treatment_effect,percent_tumor_nuclei,nfl,nfse,nt1c,nt1v,nt1d,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1,nadc.2,nfa.2,nev1.2,nev2.2,nev3.2,nevrad.2,cbvn_nlin,phn_nlin,recov_nlin,phn_npar,recov_npar,recovn_npar,ccri,cni,crni,ncho,ncre,nnaa,nlac,nlip,newdata,olddata,exclude_bc_hematoma_or_ventricle,exclude_perf,nLL,comments,perf_quant,include_perf,spec_quant,include_spec,diffu1000_quant,diffu2000_quant,exclude_bc_anat,exclude_bc_pathology,include_diffu1000,include_diffu2000
0,10B28,2723.0,t6369,6369.0,8122.0,2009-09-13,2009-09-14,Grade IV,Glioblastoma,0,0,0,1,2,1,0,0.82,183.75,1,t,NaN,1.84,2.16,1.22,0.86,0.24,1.20,0.81,1.03,1.10,1.20,1.09,NaN,NaN,NaN,NaN,NaN,NaN,0.66,0.81,84.53,0.79,84.71,NaN,2.46,2.44,-0.25,1.21,0.39,0.53,0.37,7.29,0,1,0.0,0.0,7.66,NaN,NaN,NaN,NaN,1.0,NaN,all_nas,NaN,NaN,1.0,0.0
1,11B16,2312.0,t5317,5317.0,7679.0,2008-05-04,2008-05-04,Grade IV,Glioblastoma,2,1,1,1,2,3,0,3.31,146.80,0,f,NaN,1.44,2.25,1.98,0.73,1.30,1.87,0.46,1.41,1.68,1.76,1.61,NaN,NaN,NaN,NaN,NaN,NaN,2.24,2.25,84.88,2.25,87.55,NaN,0.30,1.84,0.97,0.78,0.74,0.36,0.42,21.54,0,1,0.0,0.0,21.96,NaN,NaN,NaN,NaN,1.0,NaN,all_zeros,NaN,NaN,1.0,0.0
2,11B40,2474.0,t7408,7408.0,8701.0,2011-03-23,2011-03-24,Grade IV,Glioblastoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.97,4.13,0.57,0.95,0.28,0.08,-0.03,-2.71,0,1,NaN,NaN,-2.74,NaN,all_nas,1.0,NaN,1.0,all_nas,all_nas,1.0,1.0,0.0,0.0
3,11B43,3169.0,t7540,7540.0,8796.0,2011-06-19,2011-06-20,Grade IV,Glioblastoma,2,0,0,NaN,3,0,0,17.00,234.60,0,f,NaN,2.01,2.40,1.00,0.82,-0.01,1.39,0.65,1.12,1.11,1.16,1.13,1.49,0.83,1.12,1.31,1.39,1.4,0.58,0.67,100.00,0.60,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0.0,0.0,NaN,NaN,NaN,NaN,all_nas,0.0,NaN,NaN,NaN,NaN,1.0,1.0
4,12B16,2312.0,t5317,5317.0,7679.0,2008-05-04,2008-05-04,Grade IV,Glioblastoma,2,1,1,0,3,2,0,12.84,185.00,0,f,NaN,1.53,2.34,2.04,0.77,1.33,1.90,0.46,1.41,1.72,1.76,1.61,NaN,NaN,NaN,NaN,NaN,NaN,1.71,1.73,85.03,1.88,88.16,NaN,1.20,2.04,0.37,1.18,0.89,0.73,0.62,8.14,0,1,0.0,0.0,8.76,PP necrosis; some hyalinized BV; tissue is 2/3...,NaN,NaN,NaN,1.0,NaN,all_zeros,NaN,NaN,1.0,0.0


In [56]:
diffu1000_params = ['nadc.1', 'nfa.1', 'nev1.1', 'nev2.1', 'nev3.1', 'nevrad.1']
diffu2000_params = ['nadc.2', 'nfa.2', 'nev1.2', 'nev2.2', 'nev3.2', 'nevrad.2']


In [57]:
for idx, diffu_param in enumerate(diffu1000_params): 
    print(idx, diffu_param, diffu2000_params[idx])

0 nadc.1 nadc.2
1 nfa.1 nfa.2
2 nev1.1 nev1.2
3 nev2.1 nev2.2
4 nev3.1 nev3.2
5 nevrad.1 nevrad.2


In [58]:
## Replacing the 1000 w/ 2000: 
for idx, diffu_param in enumerate(diffu1000_params):
    recgli.loc[(recgli[diffu_param].isnull()) & recgli[diffu2000_params[idx]].notnull(), diffu_param]=recgli[diffu2000_params[idx]]
                                                                                                             
                                                                                                             
                                                                                                             
                                                                                                             

In [59]:
recgli.loc[(recgli['nadc.1'].notnull()), 'include_diffu1000']=1
recgli.loc[(recgli['nadc.1'].isnull()) , 'include_diffu1000']=0

In [60]:
Counter(recgli["include_diffu1000"])

Counter({0.0: 50, 1.0: 429})

In [61]:
recgli.to_csv('recgli_annotation6_diffuimpute.csv', index = False)

## 7. Annotate ependymomas & extraneous histologies / grades

In [62]:
recgli = pd.read_csv('recgli_annotation6_diffuimpute.csv')

In [63]:
Counter(recgli.current_hist_grade)

Counter({'Grade IV': 301,
         'Treatment Effect': 75,
         'Grade III': 88,
         'Grade II': 2,
         nan: 13})

In [64]:
Counter(recgli.current_hist_type)

Counter({'Glioblastoma': 286,
         'Treatment Effect': 72,
         'Ganglioglioma': 3,
         'Astrocytoma': 36,
         'Gliosarcoma': 15,
         'Oligodendroglioma': 45,
         'Ependymoma': 2,
         nan: 13,
         'Giant Cell Glioblastoma': 3,
         'Oligoastrocytoma': 4})

In [65]:
extraneous_histologies = ['Ganglioglioma', 'Ependymoma']

In [66]:
recgli.loc[(recgli['current_hist_grade']=="Grade II") , 'exclude_histology']=1
recgli.loc[(recgli['current_hist_type'].isin(extraneous_histologies)) , 'exclude_histology']=1

In [67]:
recgli = recgli.drop_duplicates('vialid')

In [68]:
recgli.to_csv('recgli_annotation7_extrahist.csv', index = False)

## 8. Creates outcome: 

In [69]:
recgli = pd.read_csv("recgli_annotation7_extrahist.csv")


In [70]:
recgli.head()

,vialid,b_number,t_number_x,t_number_y,sf_number,current_scan_date,current_surgery_date,current_hist_grade,current_hist_type,tumor_cell_evaluation,necrosis,mv_hyperplasia,mv_delicate_vasculature,f8_delicate,f8_simple,f8_complex,mib_1,avgerage_cells,treatment_related_ab,bx_pure_treatment_effect,percent_tumor_nuclei,nfl,nfse,nt1c,nt1v,nt1d,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1,nadc.2,nfa.2,nev1.2,nev2.2,nev3.2,nevrad.2,cbvn_nlin,phn_nlin,recov_nlin,phn_npar,recov_npar,recovn_npar,ccri,cni,crni,ncho,ncre,nnaa,nlac,nlip,newdata,olddata,exclude_bc_hematoma_or_ventricle,exclude_perf,nLL,comments,perf_quant,include_perf,spec_quant,include_spec,diffu1000_quant,diffu2000_quant,exclude_bc_anat,exclude_bc_pathology,include_diffu1000,include_diffu2000,exclude_histology
0,10B28,2723.0,t6369,6369.0,8122.0,2009-09-13,2009-09-14,Grade IV,Glioblastoma,0,0,0,1,2,1,0,0.82,183.75,1,t,NaN,1.84,2.16,1.22,0.86,0.24,1.20,0.81,1.03,1.10,1.20,1.09,NaN,NaN,NaN,NaN,NaN,NaN,0.66,0.81,84.53,0.79,84.71,NaN,2.46,2.44,-0.25,1.21,0.39,0.53,0.37,7.29,0,1,0.0,0.0,7.66,NaN,NaN,NaN,NaN,1.0,NaN,all_nas,NaN,NaN,1.0,0.0,NaN
1,11B16,2312.0,t5317,5317.0,7679.0,2008-05-04,2008-05-04,Grade IV,Glioblastoma,2,1,1,1,2,3,0,3.31,146.80,0,f,NaN,1.44,2.25,1.98,0.73,1.30,1.87,0.46,1.41,1.68,1.76,1.61,NaN,NaN,NaN,NaN,NaN,NaN,2.24,2.25,84.88,2.25,87.55,NaN,0.30,1.84,0.97,0.78,0.74,0.36,0.42,21.54,0,1,0.0,0.0,21.96,NaN,NaN,NaN,NaN,1.0,NaN,all_zeros,NaN,NaN,1.0,0.0,NaN
2,11B40,2474.0,t7408,7408.0,8701.0,2011-03-23,2011-03-24,Grade IV,Glioblastoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.97,4.13,0.57,0.95,0.28,0.08,-0.03,-2.71,0,1,NaN,NaN,-2.74,NaN,all_nas,1.0,NaN,1.0,all_nas,all_nas,1.0,1.0,0.0,0.0,NaN
3,11B43,3169.0,t7540,7540.0,8796.0,2011-06-19,2011-06-20,Grade IV,Glioblastoma,2,0,0,NaN,3,0,0,17.00,234.60,0,f,NaN,2.01,2.40,1.00,0.82,-0.01,1.39,0.65,1.12,1.11,1.16,1.13,1.49,0.83,1.12,1.31,1.39,1.4,0.58,0.67,100.00,0.60,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0.0,0.0,NaN,NaN,NaN,NaN,all_nas,0.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN
4,12B16,2312.0,t5317,5317.0,7679.0,2008-05-04,2008-05-04,Grade IV,Glioblastoma,2,1,1,0,3,2,0,12.84,185.00,0,f,NaN,1.53,2.34,2.04,0.77,1.33,1.90,0.46,1.41,1.72,1.76,1.61,NaN,NaN,NaN,NaN,NaN,NaN,1.71,1.73,85.03,1.88,88.16,NaN,1.20,2.04,0.37,1.18,0.89,0.73,0.62,8.14,0,1,0.0,0.0,8.76,PP necrosis; some hyalinized BV; tissue is 2/3...,NaN,NaN,NaN,1.0,NaN,all_zeros,NaN,NaN,1.0,0.0,NaN


In [71]:
txe_keywords = ['glia', 'yalin', 'reactive', 'reac', 'rx', 'glios', 'treatment', 'tx', 'effect', 'inflam', 'macro', 'micro']

In [72]:
recgli.shape[0]

479

In [73]:
recgli['txe_comments'] = recgli.comments.str.contains('glia|yalin|reac|rx|glios|effect|tx|inflam|macro|micro', flags=re.IGNORECASE, regex=True)

In [74]:
recgli[['comments', 'txe_comments']].tail()

,comments,txe_comments
474,NaN,NaN
475,NaN,NaN
476,"""VERY STRONG IDH1""",False
477,NaN,NaN
478,"""REACTIVE GLIA""",True


In [75]:
recgli.treatment_related_ab.value_counts()

0                 94
1                 67
Indeterminable     2
Name: treatment_related_ab, dtype: int64

In [76]:
recgli.tumor_cell_evaluation.value_counts()

3                 140
2                 127
0                 105
1                  58
Indeterminable     13
Name: tumor_cell_evaluation, dtype: int64

In [77]:
recgli['outcome'] = ['txe' if row['tumor_cell_evaluation']== "0" and 
          ((row['necrosis'] != "2") 
          and ((row['txe_comments'] == True 
              or row['treatment_related_ab'] == "1") or  
               (row["nt1c"] > 1.2 or row['nfl'] > 1.2) or 
              row['current_hist_type']=="Treatment Effect"))
              
            else 'rHGG' if (row['tumor_cell_evaluation'] == "2" or row['tumor_cell_evaluation']== "3" ) 
                     and (row['necrosis'] != "2") 
            else "unk" 
            for index, row in recgli.iterrows() ]

In [78]:
recgli['outcome'].value_counts()

rHGG    266
unk     124
txe      89
Name: outcome, dtype: int64

In [79]:
recgli.shape

(479, 71)

In [80]:
recgli.to_csv('recgli_annotation8_outcome_alt1.csv', index = False)

## 10. Annotate CEL and NEL 

In [81]:
recgli = pd.read_csv('recgli_annotation8_outcome_alt1.csv')

In [82]:
biopsy_igt_stats_info = pd.read_csv('/home/sf673542/MultiParametricMRI/GetMergeData/Aug2018/igt_stats_information_forBiopsyAnnot.csv')
biopsy_igt_stats_info.head()

,t_number,roi.label,X.CEL,X.NEL,X.NEC,sum,in_CEL,in_T2all,in_NEL,in_NEC,in_ROI
0,6369.0,10B28,NaN,NaN,NaN,NaN,0,1,1,0,1
1,5317.0,11B16,100.0,0.00,NaN,100.00,1,1,0,0,1
2,8089.0,11B35,NaN,NaN,NaN,NaN,0,0,0,0,0
3,7408.0,11B40,8.7,13.04,NaN,21.74,0,0,0,0,0
4,7540.0,11B43,0.0,28.00,NaN,28.00,0,1,1,0,1


In [83]:
biopsy_igt_stats_info = biopsy_igt_stats_info.drop_duplicates(subset = ['roi.label'])

In [84]:
biopsy_igt_stats_info = biopsy_igt_stats_info.dropna(subset = ['roi.label'])


In [85]:
biopsy_igt_stats_info = biopsy_igt_stats_info.drop(['roi.label']=="")

In [86]:
biopsy_igt_stats_info.columns =['t_number', 'vialid', 'X.CEL', 'X.NEL', 'X.NEC', 'sum', 'in_CEL',
       'in_T2all', 'in_NEL', 'in_NEC', 'in_ROI']

In [87]:
recgli_merged = recgli.merge(biopsy_igt_stats_info[['in_CEL','in_T2all', 'in_NEL', 'vialid']], how = 'left', on = ['vialid'])

In [125]:
# print(recgli_merged.in_CEL.value_counts())
# print(recgli_merged.in_T2all.value_counts())
# print(recgli_merged.in_NEL.value_counts())

In [90]:
recgli_merged['in_CEL'] = recgli_merged['in_CEL'].mask(recgli_merged['nt1c'] > 1.2, 1)
recgli_merged['in_T2all'] = recgli_merged['in_T2all'].mask(recgli_merged['nfl'] > 1.2, 1)


In [91]:
recgli_merged.loc[(recgli_merged['in_T2all']==1) & (recgli_merged['in_CEL'] != 1), 'in_NEL'] = 1
recgli_merged.loc[(recgli_merged['in_T2all']==1) & (recgli_merged['in_CEL'] == 1), 'in_NEL'] = 0
recgli_merged.loc[(recgli_merged['in_T2all']!=1) & (recgli_merged['in_CEL'] != 1), 'in_NEL'] = 0
recgli_merged.loc[(recgli_merged['in_T2all']!=1) & (recgli_merged['in_CEL'] == 1), 'in_NEL'] = 0

In [126]:
# print(recgli_merged.in_CEL.value_counts())
# print(recgli_merged.in_T2all.value_counts())
# print(recgli_merged.in_NEL.value_counts())

In [93]:
recgli_merged.loc[(recgli_merged['in_CEL']==1), 'CELorNEL'] = "CEL"
recgli_merged.loc[(recgli_merged['in_NEL']==1), 'CELorNEL'] = "NEL"
recgli_merged.loc[(recgli_merged['in_CEL']!=1) & (recgli_merged['in_NEL']!=1), 'CELorNEL'] = "unk"


In [127]:
# recgli_merged.CELorNEL.value_counts()

In [96]:
vial_manual_annot = pd.read_csv('manual_CEL_NEL_annot.csv')

In [97]:
vial_manual_annot.head()

,CELorNEL,vialid
0,CEL,13B41
1,NEL,15B32
2,NEC,25B43
3,DURA,32B43
4,SULCUS,37B17


In [98]:
recgli = recgli_merged

In [99]:
recgli_merged = recgli.merge(vial_manual_annot, how = "left", on= ["vialid" ])

In [103]:
# recgli_merged.loc[recgli_merged['vialid']=='13B41']

In [102]:
# for idx, row in recgli_merged.iterrows(): 
#     if row['CELorNEL_x'] == 'unk' and row['CELorNEL_x'] != row['CELorNEL_y']: 
#         recgli_merged.loc[idx, 'CELorNEL_x'] = row['CELorNEL_y']

In [104]:
# recgli_merged.loc[recgli_merged['vialid']=='13B41']

In [105]:
recgli_merged = recgli_merged.drop(['CELorNEL_y'], axis = 1)

In [106]:
recgli_merged.columns =['vialid', 'b_number', 't_number_x', 't_number_y', 'sf_number',
       'current_scan_date', 'current_surgery_date', 'current_hist_grade',
       'current_hist_type', 'tumor_cell_evaluation', 'necrosis',
       'mv_hyperplasia', 'mv_delicate_vasculature', 'f8_delicate', 'f8_simple',
       'f8_complex', 'mib_1', 'avgerage_cells', 'treatment_related_ab',
       'bx_pure_treatment_effect', 'percent_tumor_nuclei', 'nfl', 'nfse',
       'nt1c', 'nt1v', 'nt1d', 'nadc.1', 'nfa.1', 'nev1.1', 'nev2.1', 'nev3.1',
       'nevrad.1', 'nadc.2', 'nfa.2', 'nev1.2', 'nev2.2', 'nev3.2', 'nevrad.2',
       'cbvn_nlin', 'phn_nlin', 'recov_nlin', 'phn_npar', 'recov_npar',
       'recovn_npar', 'ccri', 'cni', 'crni', 'ncho', 'ncre', 'nnaa', 'nlac',
       'nlip', 'newdata', 'olddata', 'exclude_bc_hematoma_or_ventricle',
       'exclude_perf', 'nLL', 'comments', 'perf_quant', 'include_perf',
       'spec_quant', 'include_spec', 'diffu1000_quant', 'diffu2000_quant',
       'exclude_bc_anat', 'exclude_bc_pathology', 'include_diffu1000',
       'include_diffu2000', 'exclude_histology', 'txe_comments', 'outcome',
       'in_CEL', 'in_T2all', 'in_NEL', 'CELorNEL']

In [107]:
recgli_nelOnly = recgli_merged.loc[recgli_merged['CELorNEL']=='NEL']
recgli_nelOnly.shape

(185, 75)

In [108]:
recgli_celOnly = recgli_merged.loc[recgli_merged['CELorNEL']=='CEL']
recgli_celOnly.shape

(267, 75)

In [109]:
recgli_merged.shape

(479, 75)

In [116]:
recgli_merged.to_csv('recgli_annotation9_ROIlocation.csv', index = False)

In [117]:
recgli_nelOnly.to_csv('recgli_annotation9_NELonly.csv', index = False)
recgli_celOnly.to_csv('recgli_annotation9_CELonly.csv', index = False)


## 10. Removes things that should be excluded from analysis: 

In [118]:
recgli = pd.read_csv('recgli_annotation9_ROIlocation.csv')

In [119]:
# recgli['exclude_histology'].value_counts()

exclude b/c hematoma

In [120]:
recgli_notInHematoma = recgli.loc[recgli['exclude_bc_hematoma_or_ventricle']!=1.0]
recgli_notInHematoma.shape

(438, 75)

In [121]:
recgli_withPath = recgli_notInHematoma.loc[recgli_notInHematoma['exclude_bc_pathology']!=1]
recgli_withPath.shape

(392, 75)

In [122]:
recgli_withAnat = recgli_withPath.loc[recgli_withPath['exclude_bc_anat']!= 1]
recgli_withAnat.shape

(371, 75)

In [123]:
recgli_goodHist = recgli_withAnat.loc[recgli_withAnat['exclude_histology']!=1]
recgli_goodHist.shape

(367, 75)

In [137]:
recgli_goodHist.to_csv('recgli_annotation10_onlyGoodBiopsies_alt1.csv', index = False)

In [138]:
recgli_goodHist.CELorNEL.value_counts()

CEL          202
NEL          155
NB             6
DURA           1
NEC            1
SULCUS         1
VENTRICLE      1
Name: CELorNEL, dtype: int64

In [139]:
recgli_goodHist.outcome.value_counts()

rHGG    231
txe      73
unk      63
Name: outcome, dtype: int64

In [140]:
recgli_goodHist.outcome.isnull().value_counts()

False    367
Name: outcome, dtype: int64